# Getting started.

Import all the libraries that are to be used.
1. the pytorch framework library.
2. torchvision will also be useful to show the arch of the network.
3. resnet have to be import , Idk is it part of the torch library? gotta check.
4. numpy
5. pyplot

I'll keep adding if I see something.

In [15]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:

%reload_ext autoreload
import numpy as np
import torch
import torch.nn as nn
import torchsummary
import matplotlib.pyplot as plt
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.io import read_image
from PIL import  Image
from load_model import resnet_model

Alright , now lets try to extract resnet architecture and printout its arch using torchsummary

In [18]:

num_classes = 2
model = models.resnet18(pretrained= True)
model.fc = nn.Linear(in_features=model.fc.in_features , out_features=num_classes)
#for param in model.parameters():
#    param.requires_grad = False
model.fc.requires_grad_(True)

torchsummary.summary(model)

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            9,408
├─BatchNorm2d: 1-2                       128
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
|    └─BasicBlock: 2-1                   --
|    |    └─Conv2d: 3-1                  36,864
|    |    └─BatchNorm2d: 3-2             128
|    |    └─ReLU: 3-3                    --
|    |    └─Conv2d: 3-4                  36,864
|    |    └─BatchNorm2d: 3-5             128
|    └─BasicBlock: 2-2                   --
|    |    └─Conv2d: 3-6                  36,864
|    |    └─BatchNorm2d: 3-7             128
|    |    └─ReLU: 3-8                    --
|    |    └─Conv2d: 3-9                  36,864
|    |    └─BatchNorm2d: 3-10            128
├─Sequential: 1-6                        --
|    └─BasicBlock: 2-3                   --
|    |    └─Conv2d: 3-11                 73,728
|    |    └─BatchNorm2d: 3-12            25

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            9,408
├─BatchNorm2d: 1-2                       128
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
|    └─BasicBlock: 2-1                   --
|    |    └─Conv2d: 3-1                  36,864
|    |    └─BatchNorm2d: 3-2             128
|    |    └─ReLU: 3-3                    --
|    |    └─Conv2d: 3-4                  36,864
|    |    └─BatchNorm2d: 3-5             128
|    └─BasicBlock: 2-2                   --
|    |    └─Conv2d: 3-6                  36,864
|    |    └─BatchNorm2d: 3-7             128
|    |    └─ReLU: 3-8                    --
|    |    └─Conv2d: 3-9                  36,864
|    |    └─BatchNorm2d: 3-10            128
├─Sequential: 1-6                        --
|    └─BasicBlock: 2-3                   --
|    |    └─Conv2d: 3-11                 73,728
|    |    └─BatchNorm2d: 3-12            25

In [19]:

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

let's get some small subset of images from Imagenet to test the model.

now that we have the images, they are person, cat ,  dog , horse, car (in that order.), let's run resnet on it to see how it performs.



Now , let's get access to the r/u data. we will have a data_loader that will throw the data in batches at a time. let's take the batch size as 8 so that we can work with the dataloaders.

In [20]:
from load_data import get_data
train_loader, test_loader = get_data(batch_size =32)

In [21]:
from torch.optim import Adam
opt = Adam(model.parameters() , lr=1e-4)
criterion = nn.CrossEntropyLoss()

In [25]:
# we train our stuff here now

#run through all the epochs

# for every epoch
# get the train data from train_loader along with its label class
# perform the model on it, get the loss by comparing with truth, and then get backward the loss
# for every 10 epochs do the evaluation
opt.lr = 1e-6
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_epochs = 100
for i in range(num_epochs):
    model.train()
    losses = []
    loss_epoch = []
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        pred = model(images)
        loss = criterion( pred, labels)
        opt.zero_grad()
        loss.backward()
        opt.step()
        #print(f'training loss at epoch {i} is {loss.detach()}')
        loss_epoch.append(loss.detach())
    losses.append(sum(loss_epoch))
    print(f'training loss at epoch {i} is {losses[-1]}')
    if i%1 == 0:
        # evaluate here.
        model.eval()
        for images, labels in test_loader:
            images =images.to(device)
            labels = labels.to(device)
            pred = model(images)
            loss = criterion(pred, labels).detach()
            if i%5 == 0 : print(f'pred = {pred.argmax(dim=1)} \n label= {labels}\n')
            print(f'test_loss at epoch {i} is {loss}\n')
        model.train()


training loss at epoch 0 is 0.0018436063546687365
pred = tensor([1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0]) 
 label= tensor([1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0])

test_loss at epoch 0 is 0.08725837618112564

training loss at epoch 1 is 0.005015980452299118
test_loss at epoch 1 is 0.07837769389152527

training loss at epoch 2 is 0.2724640965461731
test_loss at epoch 2 is 0.0833655595779419

training loss at epoch 3 is 0.008372262120246887
test_loss at epoch 3 is 0.06718043982982635

training loss at epoch 4 is 0.14380982518196106
test_loss at epoch 4 is 0.047370411455631256

training loss at epoch 5 is 0.13568785786628723
pred = tensor([1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1]) 
 label= tensor([1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1])

test_loss at epoch 5 is 0.08299298584461212



KeyboardInterrupt: 

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True])

tensor([0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        0, 0, 1, 1, 0, 1, 1, 1])

# We will train only the last layer.

In [27]:

num_classes = 2
model = models.resnet18(pretrained= True)
model.fc = nn.Linear(in_features=model.fc.in_features , out_features=num_classes)
for param in model.parameters():
    param.requires_grad = False
model.fc.requires_grad_(True)

torchsummary.summary(model)

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            (9,408)
├─BatchNorm2d: 1-2                       (128)
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
|    └─BasicBlock: 2-1                   --
|    |    └─Conv2d: 3-1                  (36,864)
|    |    └─BatchNorm2d: 3-2             (128)
|    |    └─ReLU: 3-3                    --
|    |    └─Conv2d: 3-4                  (36,864)
|    |    └─BatchNorm2d: 3-5             (128)
|    └─BasicBlock: 2-2                   --
|    |    └─Conv2d: 3-6                  (36,864)
|    |    └─BatchNorm2d: 3-7             (128)
|    |    └─ReLU: 3-8                    --
|    |    └─Conv2d: 3-9                  (36,864)
|    |    └─BatchNorm2d: 3-10            (128)
├─Sequential: 1-6                        --
|    └─BasicBlock: 2-3                   --
|    |    └─Conv2d: 3-11                 (73,728)
|    |    └─BatchNorm

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            (9,408)
├─BatchNorm2d: 1-2                       (128)
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
|    └─BasicBlock: 2-1                   --
|    |    └─Conv2d: 3-1                  (36,864)
|    |    └─BatchNorm2d: 3-2             (128)
|    |    └─ReLU: 3-3                    --
|    |    └─Conv2d: 3-4                  (36,864)
|    |    └─BatchNorm2d: 3-5             (128)
|    └─BasicBlock: 2-2                   --
|    |    └─Conv2d: 3-6                  (36,864)
|    |    └─BatchNorm2d: 3-7             (128)
|    |    └─ReLU: 3-8                    --
|    |    └─Conv2d: 3-9                  (36,864)
|    |    └─BatchNorm2d: 3-10            (128)
├─Sequential: 1-6                        --
|    └─BasicBlock: 2-3                   --
|    |    └─Conv2d: 3-11                 (73,728)
|    |    └─BatchNorm

In [28]:
from torch.optim import Adam
opt = Adam(model.parameters() , lr=1e-4)
criterion = nn.CrossEntropyLoss()

In [32]:
# we train our stuff here now

#run through all the epochs

# for every epoch
# get the train data from train_loader along with its label class
# perform the model on it, get the loss by comparing with truth, and then get backward the loss
# for every 10 epochs do the evaluation
opt.lr = 1e-6
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_epochs = 100
for i in range(num_epochs):
    model.train()
    losses = []
    loss_epoch = []
    count =0
    for images, labels in train_loader:
        count += 1
        images = images.to(device)
        labels = labels.to(device)
        pred = model(images)
        loss = criterion( pred, labels)
        opt.zero_grad()
        loss.backward()
        opt.step()
        #print(f'training loss at epoch {i} is {loss.detach()}')
        loss_epoch.append(loss.detach())
    losses.append(sum(loss_epoch)/count)
    print(f'training loss at epoch {i} is {losses[-1]}')
    if i%1 == 0:
        # evaluate here.
        model.eval()
        for images, labels in test_loader:
            images =images.to(device)
            labels = labels.to(device)
            pred = model(images)
            loss = criterion(pred, labels).detach()
            if i%5 == 0 : print(f'pred = {pred.argmax(dim=1)} \n label= {labels}\n true accuracy = {sum(pred.argmax(dim=1) == labels)/len(labels)}')
            print(f'test_loss at epoch {i} is {loss}\n')
        model.train()


training loss at epoch 0 is 0.18788492679595947
pred = tensor([0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1]) 
 label= tensor([0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1])
 true accuracy = 1.0
test_loss at epoch 0 is 0.21542128920555115

training loss at epoch 1 is 0.20445393025875092
test_loss at epoch 1 is 0.22265474498271942

training loss at epoch 2 is 0.1745493859052658
test_loss at epoch 2 is 0.22470995783805847



KeyboardInterrupt: 

# Now we will train model initialized from completely random weights

In [33]:

num_classes = 2
model = models.resnet18(pretrained= False)
model.fc = nn.Linear(in_features=model.fc.in_features , out_features=num_classes)

torchsummary.summary(model)

C:\Users\Dushyant S. Udawat\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            9,408
├─BatchNorm2d: 1-2                       128
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
|    └─BasicBlock: 2-1                   --
|    |    └─Conv2d: 3-1                  36,864
|    |    └─BatchNorm2d: 3-2             128
|    |    └─ReLU: 3-3                    --
|    |    └─Conv2d: 3-4                  36,864
|    |    └─BatchNorm2d: 3-5             128
|    └─BasicBlock: 2-2                   --
|    |    └─Conv2d: 3-6                  36,864
|    |    └─BatchNorm2d: 3-7             128
|    |    └─ReLU: 3-8                    --
|    |    └─Conv2d: 3-9                  36,864
|    |    └─BatchNorm2d: 3-10            128
├─Sequential: 1-6                        --
|    └─BasicBlock: 2-3                   --
|    |    └─Conv2d: 3-11                 73,728
|    |    └─BatchNorm2d: 3-12            25

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            9,408
├─BatchNorm2d: 1-2                       128
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
|    └─BasicBlock: 2-1                   --
|    |    └─Conv2d: 3-1                  36,864
|    |    └─BatchNorm2d: 3-2             128
|    |    └─ReLU: 3-3                    --
|    |    └─Conv2d: 3-4                  36,864
|    |    └─BatchNorm2d: 3-5             128
|    └─BasicBlock: 2-2                   --
|    |    └─Conv2d: 3-6                  36,864
|    |    └─BatchNorm2d: 3-7             128
|    |    └─ReLU: 3-8                    --
|    |    └─Conv2d: 3-9                  36,864
|    |    └─BatchNorm2d: 3-10            128
├─Sequential: 1-6                        --
|    └─BasicBlock: 2-3                   --
|    |    └─Conv2d: 3-11                 73,728
|    |    └─BatchNorm2d: 3-12            25

In [34]:
from torch.optim import Adam
opt = Adam(model.parameters() , lr=1e-4)
criterion = nn.CrossEntropyLoss()

In [36]:
# we train our stuff here now

#run through all the epochs

# for every epoch
# get the train data from train_loader along with its label class
# perform the model on it, get the loss by comparing with truth, and then get backward the loss
# for every 10 epochs do the evaluation
opt.lr = 1e-6
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_epochs = 100
for i in range(num_epochs):
    model.train()
    losses = []
    loss_epoch = []
    count =0
    for images, labels in train_loader:
        count += 1
        images = images.to(device)
        labels = labels.to(device)
        pred = model(images)
        loss = criterion( pred, labels)
        opt.zero_grad()
        loss.backward()
        opt.step()
        #print(f'training loss at epoch {i} is {loss.detach()}')
        loss_epoch.append(loss.detach())
    losses.append(sum(loss_epoch)/count)
    print(f'training loss at epoch {i} is {losses[-1]}')
    if i%1 == 0:
        # evaluate here.
        model.eval()
        for images, labels in test_loader:
            images =images.to(device)
            labels = labels.to(device)
            pred = model(images)
            loss = criterion(pred, labels).detach()
            if i%5 == 0 : print(f'pred = {pred.argmax(dim=1)} \n label= {labels}\n true accuracy = {sum(pred.argmax(dim=1) == labels)/len(labels)}')
            print(f'test_loss at epoch {i} is {loss}\n')
        model.train()


training loss at epoch 0 is 0.4685856103897095
pred = tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) 
 label= tensor([1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1])
 true accuracy = 0.5
test_loss at epoch 0 is 0.741663932800293

training loss at epoch 1 is 0.6134694218635559
test_loss at epoch 1 is 1.1679657697677612

training loss at epoch 2 is 0.3706599175930023
test_loss at epoch 2 is 1.366943597793579

training loss at epoch 3 is 0.19305920600891113
test_loss at epoch 3 is 1.4533299207687378

training loss at epoch 4 is 0.1489115208387375
test_loss at epoch 4 is 1.7867956161499023

training loss at epoch 5 is 0.04631601274013519
pred = tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) 
 label= tensor([1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0])
 true accuracy = 0.5
test_loss at epoch 5 is 2.398198127746582

training loss at epoch 6 is 0.05309541895985603
test_loss at epoch 6 is 3.2148635387420654

training loss

KeyboardInterrupt: 

In [ ]:
plt.figure()
plt.plot(range(num_epochs) , losses)